# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
#Import cities file as DataFrame
cities_df = pd.read_csv("output_data/cities.csv")
cities_df.head()


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Saint Paul Harbor,57.7900,-152.4072,42.80,49,40,10.36,US,1611444735
1,1,Rikitea,-23.1203,-134.9692,79.48,63,0,7.45,PF,1611444735
2,2,Porto-Novo,6.4965,2.6036,78.80,61,100,2.30,BJ,1611444735
3,3,Atuona,-9.8000,-139.0333,78.15,80,6,18.37,PF,1611444735
4,4,Katsuura,35.1333,140.3000,37.99,100,75,20.71,JP,1611444735


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
#configure gmaps
gmaps.configure(api_key = g_key)

In [6]:
#Store latitude and longitude in locations
locations = cities_df[["Lat", "Lng"]]

#Store humidity
humidity = cities_df['Humidity']


In [7]:
#Plot Heatmap
fig = gmaps.figure()

#Add Heatmap layer to map
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10, 
                                 point_radius=1 )
#Add layer
fig.add_layer(heat_layer)

# Display figure
print(fig)


Figure(layout=FigureLayout(height='420px'))


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
weather_df = cities_df[(cities_df['Max Temp'] > 70)]
weather_df = weather_df[(weather_df['Max Temp'] < 80)]
weather_df = weather_df[(weather_df['Wind Speed'] < 10)]
weather_df = weather_df[(weather_df['Cloudiness'] == 0)]


count = len(weather_df['Max Temp'])
print(count)

weather_df.head(10)


5


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,Rikitea,-23.1203,-134.9692,79.48,63,0,7.45,PF,1611444735
60,60,Aguada de Pasajeros,22.3850,-80.8508,77.00,73,0,5.75,CU,1611444741
138,138,Makkah al Mukarramah,21.4267,39.8261,72.82,47,0,3.56,SA,1611444749
232,232,Le Port,-20.9373,55.2919,75.20,73,0,8.05,RE,1611444758
380,380,Ulundi,-28.3352,31.4162,70.57,90,0,4.25,ZA,1611444774


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:

target_radius = 5000
target_type = "lodging"
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

coordinates = ['{}, {}'.format(lat, lng) for lat, lng in zip(weather_df['Lat'], weather_df['Lng'])]
p_requests = [requests.get(base_url, params={
    'location':coordinate,
    'radius':target_radius,
    'type':target_type,
    'key':g_key
}) for coordinate in coordinates]

print(p_requests)




[<Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>]


In [10]:
places_data = [place.json() for place in p_requests]
places_data


[{'html_attributions': [],
  'results': [{'business_status': 'OPERATIONAL',
    'geometry': {'location': {'lat': -23.1206197, 'lng': -134.9687529},
     'viewport': {'northeast': {'lat': -23.1194598197085,
       'lng': -134.9673886697085},
      'southwest': {'lat': -23.1221577802915, 'lng': -134.9700866302915}}},
    'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
    'name': "Pension Maro'i",
    'opening_hours': {'open_now': True},
    'photos': [{'height': 2304,
      'html_attributions': ['<a href="https://maps.google.com/maps/contrib/111098332724302207930">luigia b.</a>'],
      'photo_reference': 'ATtYBwL_PKmGuGsIm2unYT5XEXLygWft1ZmcWeRwNgSpgGwQoBvXKCz8aY4j8yXh_sNpnhHdUTYX6rbIhJ9XqxMcgOkD6Y0pmuKnrqLPRQzlWd9n3bPahGovtfSC44jpCcREU-px0gw1S93fuydY1yyyujxKxqctujDJoxUodcgm5XmsQ801',
      'width': 4096}],
    'place_id': 'ChIJnW6Jh0aIJ54RV2HnC1r0Hjw',
    'plus_code': {'global_code': '5487V2HJ+QF'},
    'rating': 4,
    'reference': 'ChIJnW6Jh0aI

In [12]:
[place['results'][0] for place in places_data]

hotel_list = [h_attribute['results'][0]['name'] for h_attribute in places_data]
lat_list = [h_attribute['results'][0]['geometry']['location']['lat'] for h_attribute in places_data]
lng_list = [h_attribute['results'][0]['geometry']['location']['lng'] for h_attribute in places_data]
pid_list = [h_attribute['results'][0]['place_id'] for h_attribute in places_data]
lng_list


hotel_df = pd.DataFrame(data={'Hotel Name': hotel_list, 'Lat': lat_list, 'Lng': lng_list,'Place ID': pid_list }, copy=True)

hotel_df['City'] = ""
hotel_df['Country'] = ""

hotel_df


,Hotel Name,Lat,Lng,Place ID,City,Country
0,Pension Maro'i,-23.120620,-134.968753,ChIJnW6Jh0aIJ54RV2HnC1r0Hjw,,
1,Soles de Marti,22.384759,-80.849856,ChIJr99kWx9V04gRLmjr-8uXTME,,
2,"Makkah Clock Royal Tower, A Fairmont Hotel",21.418034,39.825569,ChIJN1Hy6rcEwhURanDxKxvnCIM,,
3,Villa Maido,-20.944511,55.289065,ChIJdRjMW-yIgiERENETxGOc7zI,,
4,Ekhethelo Lodge,-28.329798,31.405409,ChIJBYZpZ3Bj8B4RcQeg4-xiI5g,,


In [20]:
d_base_url = 'https://maps.googleapis.com/maps/api/place/details/json?'
a_c = 'address_component'

params = {
    'fields': a_c,
    'key':g_key
}

for index, row in hotel_df.iterrows():
    p_id  = row['Place ID']
    params['place_id'] = p_id
    
    d_requests = requests.get(d_base_url, params=params).json()
    
    itms = d_requests['result']['address_components']
    locality = None
    for itm in itms:
        if 'locality' in itm['types']:
            locality = itm['long_name']
            
            hotel_df.loc[index, 'City'] = locality
            
           
            
    itms = d_requests['result']['address_components']
    country = None
    for itm in itms:
        if 'country' in itm['types']:
            country = itm['long_name']
            
            hotel_df.loc[index, 'Country'] = country
            
               
            
hotel_df


,Hotel Name,Lat,Lng,Place ID,City,Country
0,Pension Maro'i,-23.120620,-134.968753,ChIJnW6Jh0aIJ54RV2HnC1r0Hjw,Gambier,French Polynesia
1,Soles de Marti,22.384759,-80.849856,ChIJr99kWx9V04gRLmjr-8uXTME,Aguada de Pasajeros,Cuba
2,"Makkah Clock Royal Tower, A Fairmont Hotel",21.418034,39.825569,ChIJN1Hy6rcEwhURanDxKxvnCIM,Makkah,Saudi Arabia
3,Villa Maido,-20.944511,55.289065,ChIJdRjMW-yIgiERENETxGOc7zI,,Réunion
4,Ekhethelo Lodge,-28.329798,31.405409,ChIJBYZpZ3Bj8B4RcQeg4-xiI5g,Mkhazane,South Africa


In [21]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
